In [1]:
from Alpaca_keyes import API_KEY, SECRET_KEY
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

In [2]:
from Indicators import EMA, ATR

In [3]:
api_key = API_KEY
secret_key = SECRET_KEY

symbol = "TSLA"

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key)

In [4]:
now = datetime.now(ZoneInfo("America/New_York"))

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 15, unit = TimeFrameUnit.Minute), # specify timeframe
    start = now - timedelta(days = 51),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    #limit = 2,                                               # specify limit
)

In [5]:
# timestamp is in UTC time
df = stock_historical_data_client.get_stock_bars(req).df

In [6]:
df = df.reset_index()
df['symbol'] = symbol

dataF = EMA(df).EMA_50(50)
dataF = ATR(dataF).calculate_chandelier_exit()

In [10]:
display(dataF)

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,EMA_50,ATR,Chandelier_Exit_Long,Chandelier_Exit_Short
0,TSLA,2024-11-04 11:15:00+00:00,243.6600,243.81,243.16,243.20,67299.0,1547.0,243.538648,243.200000,NaN,NaN,NaN
1,TSLA,2024-11-04 11:30:00+00:00,243.1300,243.97,243.11,243.81,62203.0,1487.0,243.569052,243.223922,NaN,NaN,NaN
2,TSLA,2024-11-04 11:45:00+00:00,243.8200,243.98,243.62,243.82,55833.0,1450.0,243.784876,243.247297,NaN,NaN,NaN
3,TSLA,2024-11-04 12:00:00+00:00,243.8700,244.00,243.20,243.59,168923.0,3341.0,243.593837,243.260737,NaN,NaN,NaN
4,TSLA,2024-11-04 12:15:00+00:00,243.6200,244.46,243.40,243.80,147145.0,2284.0,243.980495,243.281884,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262,TSLA,2024-12-24 20:45:00+00:00,464.5160,464.94,464.43,464.91,61705.0,1910.0,464.734573,450.643090,2.209991,459.870027,454.329973
2263,TSLA,2024-12-24 21:00:00+00:00,464.9237,465.13,464.75,464.90,47355.0,1766.0,464.928598,451.202185,2.113627,460.159118,455.140882
2264,TSLA,2024-12-24 21:15:00+00:00,464.9800,465.01,464.60,465.01,28342.0,1355.0,464.846876,451.743668,1.989082,460.532755,454.767245
2265,TSLA,2024-12-24 21:30:00+00:00,465.0451,465.22,464.80,464.95,33077.0,1501.0,465.029377,452.261563,1.863173,460.910482,456.389518
